In [ ]:
import ee
import folium
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import time
import functools
import operator

import subprocess
try:
    import geemap
except ImportError:
    subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap'])
    import geemap

!pip install -q geopandas
import geopandas as gpd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
ee.Authenticate()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=IDWSooVPZd5quvxE-d4JCKk06CtJwB5FOYeOOpkaTbU&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWj37ZMA-EkP3t-1e1s2yXC87xac3W6xzjrO5IqSS1bePPr_dU9ZUiU

Successfully saved authorization token.


In [ ]:
ee.Initialize()

In [ ]:
def add_ee_layer(self, ee_image_object, vis_params, name, show=True, opacity=1, min_zoom=0):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name=name,
        show=show,
        opacity=opacity,
        min_zoom=min_zoom,
        overlay=True,
        control=True
        ).add_to(self)

folium.Map.add_ee_layer = add_ee_layer


def convertUnix(x):
    """
    Converts EE.Date to Y-M-D formst
    """
    if isinstance(x, ee.Date):
        x = x.getInfo()["value"]
    
    return datetime.utcfromtimestamp(x/1000).strftime("%Y-%m-%d")# %H:%M:%S')


def mosaicByDate(collection):
    """
    Combines image tiles in a collection into a single image by day
    """ 
    ts = list(map(convertUnix, collection.reduceColumns(reducer=ee.Reducer.toList(),
                                                        selectors=["system:time_start"]
                                        ).get('list').getInfo()))

    # unique image dates in a collection in sorted order
    ts = ee.List(list({i: None for i in ts}.keys()))
    
    return ts.map(lambda x: collection.filterDate(ee.Date(x),
                                                  ee.Date(x).advance(1, "day")
                                     ).mosaic(
                                     ).set("mosaicDate", x))
    

def pointReducer(image, collection, scale, reducer):
    reducedPoints = image.reduceRegions(collection=collection,
                                        scale=scale,
                                        tileScale=2,
                                        reducer=reducer)
    
    return reducedPoints.toList(reducedPoints.size())


##### clean up function #####
def saveSampleData(data, keys, geometry, path):
    getKeys = lambda x: list(operator.itemgetter(*keys)(x["properties"]))
    data = functools.reduce(operator.iconcat, data, [])

    df = pd.DataFrame(list(map(getKeys, data)))
    df["x_coord"] = list(geometry.x)
    df["y_coord"] = list(geometry.y)
    df.columns = keys + ["x_coord", "y_coord"]
    df.to_csv(path, index=False)

In [ ]:
# load sample points and fire bounding box

fireCoords_df = gpd.read_file("/content/drive/Shareddrives/Capstone 2022/Anthony/fire selection/data/burned/sampleCoordinates.shp")
bboxCoords_df = gpd.read_file("/content/drive/Shareddrives/Capstone 2022/Anthony/fire selection/data/unburned/sampleCoordinates.shp")

bounds_df = gpd.read_file("/content/drive/Shareddrives/Capstone 2022/Anthony/fire selection/data/bounds/bounds.shp")

bounds_df["pre-fire image date"] = ["2017-10-04", "2014-07-24", "2017-08-24", "2015-09-06", "2020-07-15", "2018-10-07",
                                    "2018-07-10", "2017-10-04", "2017-07-16", "2014-07-15", "2015-07-25", "2019-10-01",
                                    "2014-09-03", "2017-07-14", "2017-09-25", "2015-07-27", "2019-09-01"]

bounds_df["post-fire image date"] = ["2017-11-05", "2014-09-10", "2017-12-14", "2015-10-24", "2020-09-01", "2018-12-26",
                                     "2018-10-14", "2017-11-05", "2017-10-04", "2014-10-03", "2015-09-20", "2019-11-18",
                                     "2014-10-21", "2017-10-18", "2017-10-27", "2015-08-19", "2020-02-24"]

In [ ]:
##### try to reduce runtime + narrow down bands + transform data #####

startTime = time.time()
imageLst = ee.List([])

fireSampleData, bboxSampleData = [], []

for fireName, preFireDate, postFireDate, geometry in bounds_df[["FIRE_NAME", "pre-fire image date", "post-fire image date", "geometry"]].values:
    t1 = time.time()
    
    firePoints = geemap.gdf_to_ee(fireCoords_df[fireCoords_df["FIRE_NAME"]==fireName])
    bboxPoints = geemap.gdf_to_ee(bboxCoords_df[bboxCoords_df["FIRE_NAME"]==fireName])
    points = ee.List([firePoints, bboxPoints])

    geometry = ee.Geometry.Rectangle(list(geometry.bounds))
    center = geometry.centroid().getInfo()["coordinates"][::-1]

    # Get pre-post fire Landsat 8 images
    preFireImage_l8 = mosaicByDate(ee.ImageCollection("LANDSAT/LC08/C02/T1_L2"
                                    ).filterBounds(geometry
                                    ).filterDate(preFireDate,
                                                 ee.Date(preFireDate).advance(1, "day")))

    postFireImage_l8 = mosaicByDate(ee.ImageCollection("LANDSAT/LC08/C02/T1_L2"
                                     ).filterBounds(geometry
                                     ).filterDate(postFireDate,
                                                  ee.Date(postFireDate).advance(1, "day")))
    
    preFireImage_l8, postFireImage_l8 = ee.Image(preFireImage_l8.get(0)), ee.Image(postFireImage_l8.get(0))

    # Calculate NBR, dNBR, and burn severity
    preFireNBR = preFireImage_l8.normalizedDifference(['SR_B5', 'SR_B7'])
    postFireNBR = postFireImage_l8.normalizedDifference(['SR_B5', 'SR_B7'])
    dNBR = (preFireNBR.subtract(postFireNBR)).multiply(1000).rename("dNBR")

    burnSeverity = dNBR.expression("(b('dNBR') > 1200) ? 5 "    # purple: high severity
                                   ":(b('dNBR') > 450) ? 4 "    # orange: moderate severity
                                   ":(b('dNBR') > 225) ? 3 "    # yellow: low severity
                                   ":(b('dNBR') > 100) ? 2 "    # green: unburned/unchanged
                                   ":(b('dNBR') > -75) ? 1 "    # brown: vegetation growth
                                   ":0"
                      ).rename("burnSeverity")
    
    # Get SRTM elevation, NLCD land cover, GRIDMET weather, and NDVI
    dem = ee.Image("NASA/NASADEM_HGT/001").select("elevation")
    nlcd2016 = ee.ImageCollection('USGS/NLCD_RELEASES/2016_REL'
                ).filter(ee.Filter.eq('system:index', '2016')).first()
    
    ndvi = postFireImage_l8.normalizedDifference(["SR_B5", "SR_B4"]).rename("NDVI")
    
    gridmet = ee.ImageCollection("IDAHO_EPSCOR/GRIDMET"
               ).filterBounds(geometry
               ).filterDate(ee.Date(postFireDate).advance(-3, "day"), postFireDate
               ).mean()

    # Merge all image bands together
    combined = postFireImage_l8.select('SR_B.'          # post-fire L8 bands 1-7
                              ).addBands(burnSeverity   # classified burn severity
                              ).addBands(dNBR           # dNBR
                              ).addBands(ndvi           # post-fire NDVI
                              ).addBands(dem            # SRTM elevation
                              ).addBands(gridmet        # all GRIDMET bands
                              ).addBands(nlcd2016       # all NLCD bands
                              ).set("FIRE_NAME", fireName)

    imageLst = imageLst.add(combined)

    # apply reducer and save results
    reducedPts = points.map(lambda x: pointReducer(image=combined,
                                                   collection=x,
                                                   scale=30,
                                                   reducer=ee.Reducer.mean()))
    lst_1, lst_2 = reducedPts.getInfo()

    fireSampleData.append(lst_1)
    bboxSampleData.append(lst_2)
    print("{} Runtime: {} minutes".format(fireName, np.round((time.time()-t1)/60, 3)))
    
print("Total Runtime: {} minutes".format(np.round((time.time()-startTime)/60, 3)))

ATLAS runtime: 0.317 minutes
BALD runtime: 0.286 minutes
BUCK runtime: 0.093 minutes
BUTTE runtime: 0.335 minutes
CALDWELL runtime: 0.331 minutes
CAMP runtime: 0.452 minutes
CARR runtime: 0.603 minutes
CASCADE runtime: 0.122 minutes
COVE runtime: 0.189 minutes
FRYING PAN runtime: 0.522 minutes
HAPPY runtime: 0.326 minutes
KINCADE runtime: 0.344 minutes
KING runtime: 0.533 minutes
OAK runtime: 0.373 minutes
REDWOOD VALLEY runtime: 0.214 minutes
ROCKY runtime: 0.236 minutes
WALKER runtime: 0.214 minutes
Total Runtime: 5.492


In [ ]:
# band names from combined image
keys = ["FIRE_NAME"] + ee.Image(imageLst.get(0)).bandNames().getInfo()

In [ ]:
# Save as csv

# # 150k fire points
# saveSampleData(data=fireSampleData,
#                keys=keys,
#                geometry=fireCoords_df["geometry"],
#                path="/content/drive/Shareddrives/Capstone 2022/Anthony/fire selection/data/burned/sampleData.csv")

# # 165k bbox points
# saveSampleData(data=bboxSampleData,
#                keys=keys,
#                geometry=bboxCoords_df["geometry"],
#                path="/content/drive/Shareddrives/Capstone 2022/Anthony/fire selection/data/unburned/sampleData.csv")

In [ ]:
# df = pd.read_csv("/content/drive/Shareddrives/Capstone 2022/Anthony/fire selection/data/unburned/sampleData.csv")
# df

,FIRE_NAME,SR_B1,SR_B2,SR_B3,SR_B4,SR_B5,SR_B6,SR_B7,burnSeverity,dNBR,NDVI,elevation,pr,rmax,rmin,sph,srad,th,tmmn,tmmx,vs,erc,eto,bi,fm100,fm1000,etr,vpd,landcover,impervious,impervious_descriptor,percent_tree_cover,rangeland_annual_herbaceous,rangeland_bare_ground,rangeland_big_sagebrush,rangeland_herbaceous,rangeland_litter,rangeland_sagebrush,rangeland_sagebrush_height,rangeland_shrub,rangeland_shrub_height,x_coord,y_coord
0,ATLAS,7273.0,7451.0,7730,7416,7395,7397,7348,1,0.107050,-0.001418,133,3.0,96.766670,58.900002,0.007217,112.599998,210.000000,279.500000,289.300018,3.233333,32.333332,1.400000,13.000000,18.100000,11.966667,1.866667,0.270000,11,0,0,0,101.0,101.0,101.0,101.0,101.0,101.0,998.0,101.0,998.0,-183748.125002,57165.624898
1,ATLAS,9116.0,9973.0,11849,14092,19003,18687,14163,1,-24.181034,0.148391,229,3.0,96.766670,58.900002,0.007217,112.599998,210.000000,279.500000,289.300018,3.233333,32.333332,1.400000,13.000000,18.100000,11.966667,1.866667,0.270000,71,0,0,0,45.0,24.0,0.0,52.0,11.0,0.0,0.0,6.0,15.0,-183775.625002,56436.874898
2,ATLAS,7744.0,7961.0,8575,8730,13556,11451,9479,1,-6.647168,0.216549,199,3.0,96.766670,58.900002,0.007217,112.599998,210.000000,279.500000,289.300018,3.233333,32.333332,1.400000,13.000000,18.100000,11.966667,1.866667,0.270000,52,0,0,33,5.0,6.0,0.0,8.0,24.0,0.0,0.0,59.0,158.0,-183775.625002,56904.374898
3,ATLAS,7368.0,7456.0,7593,7511,8461,7934,7579,1,99.826752,0.059479,480,3.0,96.300003,62.200001,0.007333,111.633331,222.666672,279.299988,288.600006,3.300000,30.333334,1.266667,12.333333,18.699999,12.566667,1.666667,0.216667,42,0,0,66,101.0,101.0,101.0,101.0,101.0,101.0,998.0,101.0,998.0,-183803.125002,54484.374898
4,ATLAS,7760.0,7843.0,8141,8340,11399,9788,8657,1,27.707573,0.154972,287,3.0,96.300003,62.200001,0.007333,111.633331,222.666672,279.299988,288.600006,3.300000,30.333334,1.266667,12.333333,18.699999,12.566667,1.666667,0.216667,71,0,0,40,22.0,4.0,0.0,25.0,10.0,0.0,0.0,41.0,141.0,-183803.125002,55804.374898
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
165056,WALKER,26517.0,28623.0,30710,31114,28666,7926,7959,0,-487.076202,-0.040950,2148,0.0,49.733334,24.733334,0.002363,193.300003,126.666664,272.600006,283.766663,3.966667,63.666668,2.733333,55.333332,7.233333,10.600000,4.033333,0.633333,52,0,0,0,0.0,30.0,0.0,19.0,21.0,0.0,0.0,6.0,99.0,-61923.125001,238253.124897
165057,WALKER,16731.0,18972.0,18296,17744,19928,7741,7691,0,-277.106781,0.057974,2149,0.0,49.733334,24.733334,0.002363,193.300003,126.666664,272.600006,283.766663,3.966667,63.666668,2.733333,55.333332,7.233333,10.600000,4.033333,0.633333,42,3,8,38,101.0,101.0,101.0,101.0,101.0,101.0,998.0,101.0,998.0,-61923.125001,238418.124897
165058,WALKER,7380.0,7450.0,7734,7510,11467,8071,7619,1,84.946373,0.208516,1529,0.0,51.799999,23.966667,0.002537,186.266678,128.333328,273.666687,285.799988,3.900000,63.000000,2.866667,54.333332,7.266666,10.833334,4.300000,0.713333,42,0,0,72,101.0,101.0,101.0,101.0,101.0,101.0,998.0,101.0,998.0,-62046.875001,219951.874897
165059,WALKER,8216.0,8419.0,9325,9293,16733,12930,10745,1,-21.977333,0.285868,1619,0.0,70.966667,22.966665,0.002667,189.600006,125.000000,270.366669,287.799988,3.900000,52.333332,2.900000,46.333332,9.700000,13.000000,4.400000,0.706667,42,0,0,46,101.0,101.0,101.0,101.0,101.0,101.0,998.0,101.0,998.0,-62046.875001,221505.624897


In [ ]:
# viz for every fire with post-fire Landsat 8 image, NLCD landcover, and classified burn severity as image layers

burnPalette = ["706c1e", "4e9d5c", "fff70b", "ff641b", "a41fd6"]

landCoverPalette = ["466b9f", "d1def8", "dec5c5", "d99282", "eb0000",
                    "ab0000", "b3ac9f", "68ab5f", "1c5f2c", "b5c58f",
                    "af963c", "ccb879", "dfdfc2", "d1d182", "a3cc51",
                    "82ba9e", "dcd939", "ab6c28", "b8d9eb", "6c9fb8"]


for name, date, geometry in bounds_df[["FIRE_NAME", "post-fire image date", "geometry"]].values:
    geometry = ee.Geometry.Rectangle(list(geometry.bounds))
    center = geometry.centroid().getInfo()["coordinates"][::-1]

    fireImage = ee.Image(imageLst.filter(ee.Filter.eq("FIRE_NAME", name)).get(0)
                 ).clip(geometry)

    m = folium.Map(location=center, zoom_start=11.25)

    m.add_ee_layer(fireImage,
                   {"bands": ["SR_B4", "SR_B3", "SR_B2"], 
                    "gamma": [0.95, 1.3, 1],
                    "min": 1000, "max": 25000},
                    "Post Fire {}".format(date))

    m.add_ee_layer(fireImage, 
                  {"bands": ["landcover"],
                   "palette": landCoverPalette},
                   "Land Cover")
    
    m.add_ee_layer(fireImage, 
                  {"bands": ["burnSeverity"],
                   "min": 0, "max": 4,
                   "palette": burnPalette},
                   "Burn Severity")
    

    m.add_child(folium.LayerControl())
    print(name)
    display(m)    
    print("\n \n")

ATLAS



 

BALD



 

BUCK



 

BUTTE



 

CALDWELL


...


 

KING



 

OAK



 

REDWOOD VALLEY



 

ROCKY



 

WALKER
